# Predict Student Performance using GamePlay

### Data Cleaning

- session_id - the ID of the session the event took place in
- index - the index of the event for the session
- elapsed_time - how much time has passed (in milliseconds) between the start of the session and when the event was recorded
- event_name - the name of the event type
- name - the event name (e.g. identifies whether a notebook_click is is opening or closing the notebook)
- level - what level of the game the event occurred in (0 to 22)
- page - the page number of the event (only for notebook-related events)
- room_coor_x - the coordinates of the click in reference to the in-game room (only for click events)
- room_coor_y - the coordinates of the click in reference to the in-game room (only for click events)
- screen_coor_x - the coordinates of the click in reference to the player’s screen (only for click events)
- screen_coor_y - the coordinates of the click in reference to the player’s screen (only for click events)
- hover_duration - how long (in milliseconds) the hover happened for (only for hover events)
- text - the text the player sees during this event
- fqid - the fully qualified ID of the event
- room_fqid - the fully qualified ID of the room the event took place in
- text_fqid - the fully qualified ID of the
- fullscreen - whether the player is in fullscreen mode
- hq - whether the game is in high-quality
- music - whether the game music is on or off
- level_group - which group of levels - and group of questions - this row belongs to (0-4, 5-12, 13-22)

In [1]:
#import libraries
import pandas as pd

In [2]:
#reading the dataset
data_frame=pd.read_csv('data/new_train_data.csv')

In [3]:
data_frame.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct
0,20090312431273200,28,28113,navigate_click,undefined,1,NaN,-587.657879,-27.916913,441.0,...,NaN,NaN,retirement_letter,tunic.historicalsociety.closet,NaN,0,0,1,0-4,1
1,20090312431273200,29,32229,notification_click,basic,1,NaN,-182.558163,-1.906501,767.0,...,NaN,Gramps is in trouble for losing papers?,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,0,0,1,0-4,1
2,20090312431273200,30,33063,notification_click,basic,1,NaN,-182.500704,-55.888296,767.0,...,NaN,This can't be right!,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,0,0,1,0-4,1
3,20090312431273200,31,34245,notification_click,basic,1,NaN,-182.486523,-55.883804,767.0,...,NaN,Gramps is a great historian!,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,0,0,1,0-4,1
4,20090312431273200,32,36433,object_click,close,1,NaN,-113.484832,241.116732,836.0,...,NaN,NaN,retirement_letter,tunic.historicalsociety.closet,NaN,0,0,1,0-4,1


## Handling outliers

In [4]:
data_frame.isnull().sum()

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page              20305214
room_coor_x        1355570
room_coor_y        1355570
screen_coor_x      1355570
screen_coor_y      1355570
hover_duration    19424403
text              12689873
fqid               6367237
room_fqid                0
text_fqid         12689791
fullscreen               0
hq                       0
music                    0
level_group              0
correct                  0
dtype: int64

#### Null values in page column

In [5]:
data_frame['page'].unique()

array([nan,  0.,  1.,  3.,  4.,  5.,  2.])

Getting the group of level values based on page number

In [6]:
data_frame.groupby('page')['event_name'].unique()

page
0.0    [notebook_click]
1.0    [notebook_click]
2.0    [notebook_click]
3.0    [notebook_click]
4.0    [notebook_click]
5.0    [notebook_click]
Name: event_name, dtype: object

In [7]:
data_frame['event_name'].unique()

array(['navigate_click', 'notification_click', 'object_click',
       'observation_click', 'cutscene_click', 'object_hover',
       'person_click', 'map_hover', 'map_click', 'checkpoint',
       'notebook_click'], dtype=object)

Page numbers are only for notebook_click (notebook related events). So, the page values for other values in event_name column are null. We need to fill null values in page column with the some other value probably -1. Since, event_name is discrete and has 0 in it. 

In [8]:
data_frame['page']=data_frame['page'].fillna(-1)

In [9]:
data_frame['page'].isnull().sum()

0

#### Null values in coordinate columns

In [10]:
#there are 4 coordinate columns such as room_coor_x, room_coor_y, screen_coor_x and screen_coor_y

The coordinate values are only for click_events

In [11]:
#shape of dataframe where room_coor_x is not null
data_frame[data_frame['room_coor_x'].isna()==False].shape

(19377008, 21)

In [12]:
#shape of dataframe where room_coor_x is null
data_frame[data_frame['room_coor_x'].isna()].shape

(1355570, 21)

In [13]:
#checking the null values for all the coordinate columns
data_frame[['room_coor_x','room_coor_y','screen_coor_x','screen_coor_y']].isnull().sum()

room_coor_x      1355570
room_coor_y      1355570
screen_coor_x    1355570
screen_coor_y    1355570
dtype: int64

In [14]:
#Below query shows that all the coordinate values are null if any coordinate value is null.

In [15]:
data_frame[data_frame['room_coor_x'].isna()&data_frame['room_coor_y'].isna()&
          data_frame['screen_coor_x'].isna()&data_frame['screen_coor_y'].isna()]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct
21,20090312431273200,49,52328,object_hover,basic,1,-1.0,NaN,NaN,NaN,...,7899.0,NaN,groupconvo,tunic.historicalsociety.entry,NaN,0,0,1,0-4,1
54,20090312431273200,82,87242,object_hover,basic,2,-1.0,NaN,NaN,NaN,...,400.0,NaN,tunic,tunic.historicalsociety.collection,NaN,0,0,1,0-4,1
59,20090312431273200,87,92242,object_hover,undefined,2,-1.0,NaN,NaN,NaN,...,3949.0,NaN,tunic.hub.slip,tunic.historicalsociety.collection,NaN,0,0,1,0-4,1
99,20090312431273200,127,135124,map_hover,basic,3,-1.0,NaN,NaN,NaN,...,234.0,NaN,tunic.historicalsociety,tunic.historicalsociety.entry,NaN,0,0,1,0-4,1
100,20090312431273200,128,135256,map_hover,basic,3,-1.0,NaN,NaN,NaN,...,17.0,NaN,tunic.kohlcenter,tunic.historicalsociety.entry,NaN,0,0,1,0-4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20732447,22100221145014656,1193,4840986,map_hover,basic,18,-1.0,NaN,NaN,NaN,...,184.0,NaN,tunic.drycleaner,tunic.capitol_2.hall,NaN,0,0,1,13-22,1
20732448,22100221145014656,1194,4841220,map_hover,basic,18,-1.0,NaN,NaN,NaN,...,84.0,NaN,tunic.drycleaner,tunic.capitol_2.hall,NaN,0,0,1,13-22,1
20732517,22100221145014656,1263,4965821,object_hover,basic,18,-1.0,NaN,NaN,NaN,...,668.0,NaN,tracks,tunic.wildlife.center,NaN,0,0,1,13-22,1
20732518,22100221145014656,1264,4966271,object_hover,undefined,18,-1.0,NaN,NaN,NaN,...,83.0,NaN,tracks.hub.deer,tunic.wildlife.center,NaN,0,0,1,13-22,1


Let us replace all the coordinate values with 0. Since, the coordinate value pair will 0,0.
Before that, we will check whether there is (0,0) pair in coordinate columns.

In [16]:
#checking for room coordinates
data_frame[(data_frame['room_coor_x']==0)&(data_frame['room_coor_y']==0)]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct


In [17]:
#checking for screen coordinates
data_frame[(data_frame['screen_coor_x']==0)&(data_frame['screen_coor_y']==0)]

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct


We can conclude that there is no certain pair that is (0,0). So, we can replace 0 in all the null values in coordinate columns

In [18]:
data_frame['room_coor_x']=data_frame['room_coor_x'].fillna(0)
data_frame['room_coor_y']=data_frame['room_coor_y'].fillna(0)
data_frame['screen_coor_x']=data_frame['screen_coor_x'].fillna(0)
data_frame['screen_coor_y']=data_frame['screen_coor_y'].fillna(0)

In [19]:
#re-checking the null values for all the coordinate columns
data_frame.isnull().sum()

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page                     0
room_coor_x              0
room_coor_y              0
screen_coor_x            0
screen_coor_y            0
hover_duration    19424403
text              12689873
fqid               6367237
room_fqid                0
text_fqid         12689791
fullscreen               0
hq                       0
music                    0
level_group              0
correct                  0
dtype: int64

Handling missing values for hover_duration column

In [20]:
0 in data_frame['hover_duration'].unique()

True

Let us replace 0 with the null values in the hover_duration column since, there will be no hover event to calculate the time

In [21]:
data_frame['hover_duration']=data_frame['hover_duration'].fillna(0)

In [22]:
data_frame.isnull().sum()

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page                     0
room_coor_x              0
room_coor_y              0
screen_coor_x            0
screen_coor_y            0
hover_duration           0
text              12689873
fqid               6367237
room_fqid                0
text_fqid         12689791
fullscreen               0
hq                       0
music                    0
level_group              0
correct                  0
dtype: int64

In [23]:
#the remaining columns are of object type

Replacing the null values for fqid, text_fqid is related to each other.

In [24]:
#getting the data_frame records with specific columns
fqid_data=data_frame[['level','index','event_name','name','fqid','room_fqid','text_fqid']].copy()

In [25]:
#filling the null values for extracted_fqid
#creating the new column extracted_fqid which will store the fqid column's values. 
fqid_data['extracted_fqid']=fqid_data['fqid']
#checking the records where fqid is null and text_fqid is not and storing
text_fqid_nn=fqid_data[fqid_data['fqid'].isna()&(fqid_data['text_fqid'].isna()==False)]

In [26]:
#assigning the values for extracted_fqid column with the extracted_fqid column's value in 
#text_fqid_nn dataframe with the text_fqid_nn index values.
fqid_data['extracted_fqid'].iloc[text_fqid_nn.index]=text_fqid_nn['extracted_fqid']

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [27]:
fqid_data.head()

,level,index,event_name,name,fqid,room_fqid,text_fqid,extracted_fqid
0,1,28,navigate_click,undefined,retirement_letter,tunic.historicalsociety.closet,NaN,retirement_letter
1,1,29,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN
2,1,30,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN
3,1,31,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN
4,1,32,object_click,close,retirement_letter,tunic.historicalsociety.closet,NaN,retirement_letter


In [28]:
#creating a new column named extracted_text_fqid and storing text_fqid column values.
fqid_data['extracted_text_fqid']=fqid_data['text_fqid']

In [29]:
#getting the index values of fqid_data where text_fqid has null values and extracted_fqid. doesn't null values..
ex_text_fqid_index=fqid_data[(fqid_data['text_fqid'].isna())&(fqid_data['extracted_fqid'].isna()==False)].index

In [30]:
#assigning the values i.e combination of room_fqid and extracted_fqid column values to extracted_text_fqid at the index values taken from above. 
fqid_data['extracted_text_fqid'].iloc[ex_text_fqid_index]=fqid_data['room_fqid'].iloc[ex_text_fqid_index]+'.'+fqid_data['extracted_fqid'].iloc[ex_text_fqid_index]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [31]:
#creating a new column combine by joining the values in event_name, name, room_fqid and level as a single string. 
fqid_data['combine']=fqid_data['event_name']+fqid_data['name']+fqid_data['room_fqid']+fqid_data['level'].astype(str)

In [32]:
#creating a new combine_copy and storing extracted_fqid column's values.
fqid_data['combine_copy']=fqid_data['extracted_fqid']

In [33]:
#taking limited columns event_name, name, room_fqid, extracted_text_fqid, 'extracted_fqid', level 
#to replace the null values of fqid column
sample_data=fqid_data[['event_name','name','room_fqid','extracted_text_fqid','extracted_fqid','level']]
#getting the records from the above dataframe where the extracted_fqid is null and dropping the duplicates in it.
sample_data_uni=sample_data[sample_data['extracted_fqid'].isna()==False].drop_duplicates()

In [34]:
#shape of above dataframe after dropping duplicates
sample_data_uni.shape

(2226, 6)

In [35]:
#creating a dictionary to store the values of fqid
fqid_dict=dict()
#combining all the string values of event_name, name, room_fqid and level and storing in a new columm 'combine'
sample_data_uni['combine']=sample_data_uni['event_name']+sample_data_uni['name']+sample_data_uni['room_fqid']+sample_data_uni['level'].astype(str)
#iterating through the unique values of combine column 
for comb in sample_data_uni.drop_duplicates()['combine'].unique():
    #checking whether the extracted_fqid has only one unique value through each combine value
    if len(sample_data_uni[sample_data_uni['combine']==comb]['extracted_fqid'].unique())==1:
        #if yes, then printing the combine value and the extracted_fqid value
        print(comb)
        print('---------')
        print(sample_data_uni[sample_data_uni['combine']==comb]['extracted_fqid'].unique())
        print('----------------------------------------------------------------------')
        #assigning the extracted_fqid value to the fqid_dict where combine is the key
        fqid_dict[comb]=sample_data_uni[sample_data_uni['combine']==comb]['extracted_fqid'].unique()[0]

object_clickclosetunic.historicalsociety.closet1
---------
['retirement_letter']
----------------------------------------------------------------------
observation_clickbasictunic.historicalsociety.basement1
---------
['janitor']
----------------------------------------------------------------------
cutscene_clickbasictunic.historicalsociety.entry1
---------
['groupconvo']
----------------------------------------------------------------------
object_clickclosetunic.historicalsociety.entry2
---------
['directory']
----------------------------------------------------------------------
cutscene_clickbasictunic.historicalsociety.collection2
---------
['cs']
----------------------------------------------------------------------
object_hoverundefinedtunic.historicalsociety.collection2
---------
['tunic.hub.slip']
----------------------------------------------------------------------
object_clickclosetunic.historicalsociety.collection2
---------
['tunic']
-------------------------------------

navigate_clickundefinedtunic.historicalsociety.frontdesk18
---------
['tostacks']
----------------------------------------------------------------------
object_clickbasictunic.historicalsociety.entry3
---------
['directory']
----------------------------------------------------------------------
object_clickclosetunic.humanecology.frontdesk11
---------
['businesscards']
----------------------------------------------------------------------
observation_clickbasictunic.historicalsociety.closet_dirty18
---------
['photo']
----------------------------------------------------------------------
person_clickbasictunic.drycleaner.frontdesk18
---------
['worker']
----------------------------------------------------------------------
person_clickbasictunic.library.frontdesk18
---------
['worker']
----------------------------------------------------------------------
object_clickclosetunic.library.microfiche18
---------
['reader']
-------------------------------------------------------------------

object_clickclosetunic.historicalsociety.stacks18
---------
['journals']
----------------------------------------------------------------------
object_hoverundefinedtunic.historicalsociety.collection6
---------
['tunic.hub.slip']
----------------------------------------------------------------------
object_hoverbasictunic.historicalsociety.collection11
---------
['tunic']
----------------------------------------------------------------------
object_hoverundefinedtunic.historicalsociety.collection11
---------
['tunic.hub.slip']
----------------------------------------------------------------------
object_clickclosetunic.historicalsociety.collection11
---------
['tunic']
----------------------------------------------------------------------
observation_clickbasictunic.library.microfiche9
---------
['block_0']
----------------------------------------------------------------------
object_hoverundefinedtunic.historicalsociety.collection7
---------
['tunic.hub.slip']
------------------------

object_clickbasictunic.kohlcenter.halloffame13
---------
['plaque']
----------------------------------------------------------------------
object_hoverbasictunic.kohlcenter.halloffame13
---------
['plaque']
----------------------------------------------------------------------
object_clickclosetunic.kohlcenter.halloffame13
---------
['plaque']
----------------------------------------------------------------------
object_clickclosetunic.kohlcenter.halloffame7
---------
['plaque']
----------------------------------------------------------------------
object_hoverbasictunic.humanecology.frontdesk12
---------
['businesscards']
----------------------------------------------------------------------
object_hoverbasictunic.kohlcenter.halloffame16
---------
['plaque']
----------------------------------------------------------------------
object_clickclosetunic.kohlcenter.halloffame16
---------
['plaque']
----------------------------------------------------------------------
object_clickclosetun

In [36]:
test_df=sample_data_uni

In [37]:
len(fqid_dict)

298

In [38]:
len(test_df['combine'].unique())

744

In [39]:
len(test_df['combine'])

2226

In [40]:
#getting the index value from the fqid_data after mapping the values with the values in fqid_dict
#where extracted_fqid is null
null_fqid_data_ind=fqid_data[fqid_data['extracted_fqid'].isna()]['combine'].map(fqid_dict).index

In [41]:
#assigning the values to the combine_copy at the null_fqid_data_ind index values with the 
#combine values by mapping with fqid_dict dictionary where extracted_fqid column has null values
fqid_data['combine_copy'].iloc[null_fqid_data_ind]=fqid_data[fqid_data['extracted_fqid'].isna()]['combine'].map(fqid_dict)

In [42]:
fqid_data.head()

,level,index,event_name,name,fqid,room_fqid,text_fqid,extracted_fqid,extracted_text_fqid,combine,combine_copy
0,1,28,navigate_click,undefined,retirement_letter,tunic.historicalsociety.closet,NaN,retirement_letter,tunic.historicalsociety.closet.retirement_letter,navigate_clickundefinedtunic.historicalsociety...,retirement_letter
1,1,29,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN
2,1,30,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN
3,1,31,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN
4,1,32,object_click,close,retirement_letter,tunic.historicalsociety.closet,NaN,retirement_letter,tunic.historicalsociety.closet.retirement_letter,object_clickclosetunic.historicalsociety.closet1,retirement_letter


In [43]:
#getting the index of fqid_data where extracted_text_fqid is null and combine_copy is not null
cc_nn_index=fqid_data[fqid_data['extracted_text_fqid'].isna()&(fqid_data['combine_copy'].isna()==False)].index

In [44]:
#assigning the values of extracted_text_fqid to the newly created column extracted_text_fqid_copy 
fqid_data['extracted_text_fqid_copy']=fqid_data['extracted_text_fqid']

In [45]:
#assigning the combined values of room_fqid, combine_copy with the index values in cc_nn_index
#to the extracted_text_fqid_copy column at the index cc_nn_index
fqid_data['extracted_text_fqid_copy'].iloc[cc_nn_index]=fqid_data['room_fqid'].iloc[cc_nn_index]+'.'+fqid_data['combine_copy'].iloc[cc_nn_index]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [46]:
fqid_data.isnull().sum()

level                              0
index                              0
event_name                         0
name                               0
fqid                         6367237
room_fqid                          0
text_fqid                   12689791
extracted_fqid               6367237
extracted_text_fqid          5946761
combine                            0
combine_copy                 6303146
extracted_text_fqid_copy     5882670
dtype: int64

In [47]:
fqid_data.head()

,level,index,event_name,name,fqid,room_fqid,text_fqid,extracted_fqid,extracted_text_fqid,combine,combine_copy,extracted_text_fqid_copy
0,1,28,navigate_click,undefined,retirement_letter,tunic.historicalsociety.closet,NaN,retirement_letter,tunic.historicalsociety.closet.retirement_letter,navigate_clickundefinedtunic.historicalsociety...,retirement_letter,tunic.historicalsociety.closet.retirement_letter
1,1,29,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.closet.retirement_lett...
2,1,30,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.closet.retirement_lett...
3,1,31,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.closet.retirement_lett...
4,1,32,object_click,close,retirement_letter,tunic.historicalsociety.closet,NaN,retirement_letter,tunic.historicalsociety.closet.retirement_letter,object_clickclosetunic.historicalsociety.closet1,retirement_letter,tunic.historicalsociety.closet.retirement_letter


In [48]:
#checking the records where extracted_text_fqid_copy is not null and combine_copy is null
fqid_data[(fqid_data['extracted_text_fqid_copy'].isna()==False)&(fqid_data['combine_copy'].isna())]

,level,index,event_name,name,fqid,room_fqid,text_fqid,extracted_fqid,extracted_text_fqid,combine,combine_copy,extracted_text_fqid_copy
1,1,29,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.closet.retirement_lett...
2,1,30,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.closet.retirement_lett...
3,1,31,notification_click,basic,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,NaN,tunic.historicalsociety.closet.retirement_lett...,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.closet.retirement_lett...
57,2,85,notification_click,basic,NaN,tunic.historicalsociety.collection,tunic.historicalsociety.collection.tunic.slip,NaN,tunic.historicalsociety.collection.tunic.slip,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.collection.tunic.slip
58,2,86,notification_click,basic,NaN,tunic.historicalsociety.collection,tunic.historicalsociety.collection.tunic.slip,NaN,tunic.historicalsociety.collection.tunic.slip,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.collection.tunic.slip
...,...,...,...,...,...,...,...,...,...,...,...,...
20731908,10,609,notification_click,basic,NaN,tunic.library.frontdesk,tunic.library.frontdesk.wellsbadge.hub,NaN,tunic.library.frontdesk.wellsbadge.hub,notification_clickbasictunic.library.frontdesk10,NaN,tunic.library.frontdesk.wellsbadge.hub
20732081,11,782,notification_click,basic,NaN,tunic.historicalsociety.stacks,tunic.historicalsociety.stacks.journals.pic_2....,NaN,tunic.historicalsociety.stacks.journals.pic_2....,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.stacks.journals.pic_2....
20732082,11,783,notification_click,basic,NaN,tunic.historicalsociety.stacks,tunic.historicalsociety.stacks.journals.pic_2....,NaN,tunic.historicalsociety.stacks.journals.pic_2....,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.stacks.journals.pic_2....
20732083,11,784,notification_click,basic,NaN,tunic.historicalsociety.stacks,tunic.historicalsociety.stacks.journals.pic_2....,NaN,tunic.historicalsociety.stacks.journals.pic_2....,notification_clickbasictunic.historicalsociety...,NaN,tunic.historicalsociety.stacks.journals.pic_2....


In [49]:
#assigning the values in combine_copy of fqid_data to fqid column in data_frame
data_frame['fqid']=fqid_data['combine_copy']

In [50]:
fqid_data['extracted_text_fqid_copy'].str.split('.').str[3:]

0                [retirement_letter]
1           [retirement_letter, hub]
2           [retirement_letter, hub]
3           [retirement_letter, hub]
4                [retirement_letter]
                      ...           
20732573             [wells, nodeer]
20732574             [wells, nodeer]
20732575             [wells, nodeer]
20732576             [wells, nodeer]
20732577             [wells, nodeer]
Name: extracted_text_fqid_copy, Length: 20732578, dtype: object

In [51]:
#assigning the values in extracted_text_fqid_copy of fqid_data to text_fqid column in data_frame
data_frame['text_fqid']=fqid_data['extracted_text_fqid_copy']

In [52]:
data_frame.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,...,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct
0,20090312431273200,28,28113,navigate_click,undefined,1,-1.0,-587.657879,-27.916913,441.0,...,0.0,NaN,retirement_letter,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_letter,0,0,1,0-4,1
1,20090312431273200,29,32229,notification_click,basic,1,-1.0,-182.558163,-1.906501,767.0,...,0.0,Gramps is in trouble for losing papers?,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,0,0,1,0-4,1
2,20090312431273200,30,33063,notification_click,basic,1,-1.0,-182.500704,-55.888296,767.0,...,0.0,This can't be right!,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,0,0,1,0-4,1
3,20090312431273200,31,34245,notification_click,basic,1,-1.0,-182.486523,-55.883804,767.0,...,0.0,Gramps is a great historian!,NaN,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_lett...,0,0,1,0-4,1
4,20090312431273200,32,36433,object_click,close,1,-1.0,-113.484832,241.116732,836.0,...,0.0,NaN,retirement_letter,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_letter,0,0,1,0-4,1


In [53]:
#re-checking the null values in the data frame
data_frame.isnull().sum()

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page                     0
room_coor_x              0
room_coor_y              0
screen_coor_x            0
screen_coor_y            0
hover_duration           0
text              12689873
fqid               6303146
room_fqid                0
text_fqid          5882670
fullscreen               0
hq                       0
music                    0
level_group              0
correct                  0
dtype: int64

In [54]:
#replacing the remaining null values in the fqid and text_fqid column with the string value 'Missing'

In [55]:
#checking whether missing exists in fqid column
'missing' in data_frame['fqid'].unique()

False

In [56]:
'Missing' in data_frame['fqid'].unique()

False

In [57]:
#checking whether missing exists in text_fqid column
'missing' in data_frame['text_fqid'].unique()

False

In [58]:
'Missing' in data_frame['text_fqid'].unique()

False

In [59]:
#replace the null values in fqid column with 'Missing' value

In [60]:
data_frame['fqid']=data_frame['fqid'].fillna('Missing')

In [61]:
data_frame['fqid']=data_frame['fqid'].str.split('.').str[0]

In [62]:
#replace the null values in text_fqid column with 'Missing' value

In [63]:
data_frame['text_fqid']=data_frame['text_fqid'].fillna('Missing')

In [64]:
data_frame['text_fqid']=data_frame['room_fqid']+'.'+data_frame['fqid']

In [65]:
#re-check the null value count
data_frame.isnull().sum()

session_id               0
index                    0
elapsed_time             0
event_name               0
name                     0
level                    0
page                     0
room_coor_x              0
room_coor_y              0
screen_coor_x            0
screen_coor_y            0
hover_duration           0
text              12689873
fqid                     0
room_fqid                0
text_fqid                0
fullscreen               0
hq                       0
music                    0
level_group              0
correct                  0
dtype: int64

In [66]:
#removing the redudant column such as 'text'

In [67]:
final_data_frame=data_frame.drop(['text'],axis=1)

In [68]:
final_data_frame.head()

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,correct
0,20090312431273200,28,28113,navigate_click,undefined,1,-1.0,-587.657879,-27.916913,441.0,303.0,0.0,retirement_letter,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_letter,0,0,1,0-4,1
1,20090312431273200,29,32229,notification_click,basic,1,-1.0,-182.558163,-1.906501,767.0,305.0,0.0,Missing,tunic.historicalsociety.closet,tunic.historicalsociety.closet.Missing,0,0,1,0-4,1
2,20090312431273200,30,33063,notification_click,basic,1,-1.0,-182.500704,-55.888296,767.0,359.0,0.0,Missing,tunic.historicalsociety.closet,tunic.historicalsociety.closet.Missing,0,0,1,0-4,1
3,20090312431273200,31,34245,notification_click,basic,1,-1.0,-182.486523,-55.883804,767.0,359.0,0.0,Missing,tunic.historicalsociety.closet,tunic.historicalsociety.closet.Missing,0,0,1,0-4,1
4,20090312431273200,32,36433,object_click,close,1,-1.0,-113.484832,241.116732,836.0,62.0,0.0,retirement_letter,tunic.historicalsociety.closet,tunic.historicalsociety.closet.retirement_letter,0,0,1,0-4,1


In [69]:
final_data_frame.shape

(20732578, 20)

In [70]:
final_data_frame.to_csv('data/clean_train_data.csv',index=False)